In [2]:
%pip install numpy
%pip install pandas
%pip install matplotlib

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# reading and filtering dataset 2 bankrupcty data
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)
BNK_DATA = pd.read_csv(
    './data/Qualitative_Bankruptcy.data.txt', 
    sep=',',
    names=['industrial risk', 'management risk', 'financial flexibility', 'credibility', 'competitiveness', 'operating risk', 'class'])

# mapping data positive: 1, average: 0.5, negative: 0 ... classification not bankrupt: 0, bankrupt: 1
for col in BNK_DATA:
    BNK_DATA[col] = BNK_DATA[col].replace({
        'P': 1, 
        'A': 0, 
        'N': -1,
        'NB': 0,
        'B': 1})

print(BNK_DATA)
BNK_DATA = BNK_DATA.to_numpy()

# shuffling bnk_data across axis 1 before making train validation test splits
np.random.shuffle(BNK_DATA)

# 60:20:20 split
BNK_TRAIN = BNK_DATA[:150, :]           # 150 samples for training
BNK_VALIDATION = BNK_DATA[150:200, :]   # 50 samples for validation / hyperparameter tuning
BNK_TEST = BNK_DATA[200:, :]            # 50 samples for testing
BNK_TRAIN

     industrial risk  management risk  financial flexibility  credibility  competitiveness  operating risk  class
0                  1                1                      0            0                0               1      0
1                 -1               -1                      0            0                0              -1      0
2                  0                0                      0            0                0               0      0
3                  1                1                      1            1                1               1      0
4                 -1               -1                      1            1                1              -1      0
..               ...              ...                    ...          ...              ...             ...    ...
245               -1               -1                      0           -1               -1              -1      1
246                1               -1                     -1           -1               

array([[ 0, -1, -1, ...,  1,  1,  0],
       [ 0,  0,  0, ...,  1,  0,  0],
       [ 1, -1, -1, ..., -1,  1,  1],
       ...,
       [ 1,  0,  1, ...,  1, -1,  0],
       [ 0, -1, -1, ..., -1,  0,  1],
       [ 0,  1, -1, ..., -1, -1,  1]], dtype=int64)

In [4]:
# bankrupcty dataset stats
n, d = BNK_DATA.shape
print(f'instances: {n}\nfeatures: {d}\nmean: {np.mean(BNK_DATA[:, :-1].flatten())}\nmedian: {np.median(BNK_DATA.flatten())}')
print(f'bankrupt: {sum(BNK_DATA[:, 6])}\nnot bankrupt: {250-sum(BNK_DATA[:, 6])}')

n_train, d_train = BNK_TRAIN.shape
n_test, d_test = BNK_TEST.shape
n_validation, d_validation = BNK_VALIDATION.shape
# adding ones column for bias
BNK_TRAIN = np.column_stack((np.ones(n_train),BNK_TRAIN))
BNK_TEST = np.column_stack((np.ones(n_test),BNK_TEST))
BNK_VALIDATION = np.column_stack((np.ones(n_validation),BNK_VALIDATION))

instances: 250
features: 7
mean: -0.12666666666666668
median: 0.0
bankrupt: 107
not bankrupt: 143


In [5]:
# logistic squishification function
logistic = lambda z : 1. / (1 + np.exp(-z))

#cross entropy loss function 
# w: weight column vector   D x 1
# x: feature matrix         N x D
# y: expected labels        N x 1
# yh: predicted labels       N x 1
# yh = x @ w
def loss(x, y, w):
    N, D = x.shape                                                       
    z = np.dot(x, w)
    return np.mean(y * np.log1p(np.exp(-z)) + (1-y) * np.log1p(np.exp(z)))  #log1p calculates log(1+x) to remove floating point inaccuracies 

# gradient descent
# w: weight column vector   D x 1
# x: feature matrix         N x D
# y: expected labels        N x 1
# yh: predicted labels       N x 1
def gradient(w, x, y):
    n, d = x.shape
    yh = logistic(np.dot(x, w))
    return np.dot(x.T, yh - y) / n

def fit(x, y, learning_rate: int=0.09, epsilon: int=0.001, max_iters: int=1000):
    n, d = x.shape
    w = np.zeros(d)
    g = np.inf 
    t = 0
    # the code snippet below is for gradient descent
    while np.linalg.norm(g) > epsilon and t < max_iters:
        g = gradient(w, x, y)
        w = w - learning_rate * g 
        t += 1

    return w
    
def predict(w, x):
    yh = logistic(np.dot(x, w))
    return np.array([round(num, 2) for num in yh])

x = BNK_TRAIN[:, :7]
y = BNK_TRAIN[:, 7]
# print(x)
# print(y)


w = fit(x, y)

x = BNK_TEST[:, :7]
yh = predict(w, x)
y = BNK_TEST[:, 7]

print(f'    y: yh\n{np.column_stack((y,yh))}')
print(f'loss: {loss(x,y,w)}')

    y: yh
[[0.   0.02]
 [0.   0.07]
 [1.   1.  ]
 [0.   0.34]
 [0.   0.  ]
 [0.   0.  ]
 [0.   0.  ]
 [0.   0.03]
 [1.   0.99]
 [0.   0.  ]
 [0.   0.  ]
 [1.   0.99]
 [0.   0.  ]
 [1.   0.96]
 [1.   0.99]
 [0.   0.  ]
 [1.   0.93]
 [1.   0.99]
 [0.   0.25]
 [1.   1.  ]
 [1.   0.99]
 [0.   0.01]
 [1.   0.98]
 [0.   0.01]
 [0.   0.01]
 [1.   0.99]
 [0.   0.01]
 [1.   0.98]
 [0.   0.03]
 [1.   0.99]
 [0.   0.18]
 [1.   1.  ]
 [1.   0.98]
 [1.   0.99]
 [1.   1.  ]
 [1.   0.99]
 [0.   0.  ]
 [1.   0.33]
 [0.   0.  ]
 [1.   1.  ]
 [0.   0.01]
 [0.   0.01]
 [0.   0.  ]
 [1.   1.  ]
 [1.   0.99]
 [0.   0.01]
 [1.   0.99]
 [0.   0.18]
 [1.   0.8 ]
 [0.   0.03]]
loss: 0.059083249406021415
